In [38]:
from synthetix_v3.base_perps import BasePerps
import matplotlib.pyplot as plt
import polars as pl
import matplotlib.dates as mdates

In [11]:
base_perp = BasePerps()

df = base_perp.get_account_liquidations()

In [43]:
polars_df = pl.from_pandas(df[['accountLiquidateds_timestamp', 'accountLiquidateds_fullyLiquidated']])

In [44]:
polars_df = polars_df.with_columns([
    pl.from_epoch('accountLiquidateds_timestamp').dt.round('1d').alias('datetime')
])

In [47]:
polars_df.group_by('datetime').agg([pl.count('accountLiquidateds_fullyLiquidated')]).sort('datetime', descending=True).head(5)

datetime,accountLiquidateds_fullyLiquidated
datetime[μs],u32
2023-10-14 00:00:00,26
2023-10-13 00:00:00,18
2023-10-12 00:00:00,7
2023-10-11 00:00:00,10
2023-10-10 00:00:00,5


In [48]:
grouped_df = polars_df.group_by('datetime').agg([pl.count('accountLiquidateds_fullyLiquidated')]).sort('datetime', descending=True).to_pandas()

In [49]:
# Pivot the DataFrame to create a stacked bar plot
pivot_df = grouped_df.pivot(index='datetime', columns='accountLiquidateds_fullyLiquidated', values='count').fillna(0)

# Plotting
fig, ax = plt.subplots(figsize=(12, 6))
pivot_df.plot(kind='bar', stacked=True, ax=ax)

ax.set_xlabel("Date")
ax.set_ylabel("# of Trades")
ax.set_title("Daily Trades by Market")

# Format the x-axis labels to display only the date
date_format = mdates.DateFormatter("%Y-%m-%d")
ax.xaxis.set_major_formatter(date_format)

plt.xticks(range(len(pivot_df.index)), [ts.strftime('%Y-%m-%d') for ts in pivot_df.index], rotation=45)
plt.legend(title="Market Symbols")
plt.show()

KeyError: 'markets_marketSymbol'